In [7]:
import collections.abc

# Define the M3System class to hold the parameters and modulus
# This encapsulates the specific algebraic system (V, *)
class M3System:
    def __init__(self, A, B, C, D, E, modulus):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        self.E = E
        self.modulus = modulus

    # Override __repr__ for better readability of the system object
    def __repr__(self):
        return (f"M3System(A={self.A}, B={self.B}, C={self.C}, D={self.D}, E={self.E}, "
                f"modulus={self.modulus})")

# Define the M3Element class to represent vectors within a specific M3System
class M3Element:
    def __init__(self, value: list[int], system: M3System):
        if not isinstance(value, collections.abc.Sequence) or len(value) != 3:
            raise ValueError("Value must be a list or tuple of 3 integers.")
        if not isinstance(system, M3System):
            raise TypeError("System must be an instance of M3System.")

        self.system = system
        self.value = [x % self.system.modulus for x in value]

    # Standard vector addition for the underlying vector space
    def __add__(self, other):
        if not isinstance(other, M3Element) or self.system != other.system:
            return NotImplemented # Or raise ValueError for system mismatch
        return M3Element([(x + y) % self.system.modulus for x, y in zip(self.value, other.value)], self.system)

    # Standard vector subtraction
    def __sub__(self, other):
        if not isinstance(other, M3Element) or self.system != other.system:
            return NotImplemented
        return M3Element([(x - y) % self.system.modulus for x, y in zip(self.value, other.value)], self.system)

    # Standard unary negation
    def __neg__(self):
        return M3Element([(-x) % self.system.modulus for x in self.value], self.system)

    # The core binary operation '*' as defined in the article
    # Corresponds to (ab) in the article
    def __mul__(self, other):
        if not isinstance(other, M3Element) or self.system != other.system:
            return NotImplemented # Or raise ValueError for system mismatch

        # Components of vector 'a' (self)
        a0, a1, a2 = self.value
        # Components of vector 'b' (other)
        b0, b1, b2 = other.value
        
        # Parameters of the M3System
        A, B, C, D, E = self.system.A, self.system.B, self.system.C, self.system.D, self.system.E
        N = self.system.modulus

        # Component-wise definition of (ab)_i based on the article's K^3 formula
        # (ab)_0 = a_0 + b_0 + a_0 b_0 + A a_1 b_1 + C a_2 b_1 + B a_2 b_2
        r0 = (a0 + b0 + a0 * b0 + A * a1 * b1 + C * a2 * b1 + B * a2 * b2) % N
        
        # (ab)_1 = a_1 + b_1 + a_1 b_0 + a_0 b_1 + D a_1 b_1 + E a_1 b_2
        r1 = (a1 + b1 + a1 * b0 + a0 * b1 + D * a1 * b1 + E * a1 * b2) % N
        
        # (ab)_2 = a_2 + b_2 + a_2 b_0 + a_0 b_2 + D a_2 b_1 + E a_2 b_2
        r2 = (a2 + b2 + a2 * b0 + a0 * b2 + D * a2 * b1 + E * a2 * b2) % N

        return M3Element([r0, r1, r2], self.system)

    # Implements exponentiation a^n (repeated application of '*')
    # Uses exponentiation by squaring for efficiency
    def __pow__(self, exponent: int):
        if not isinstance(exponent, int) or exponent < 0:
            raise ValueError("Exponent must be a non-negative integer.")
        
        # The neutral element 'e' (multiplicative identity) as defined in the article (0,0,0)
        # a * e = e * a = a
        identity_element = M3Element([0, 0, 0], self.system) 

        if exponent == 0:
            return identity_element

        # Start with the base vector 'a'
        base = self 
        # Initialize result with the identity element
        result = identity_element

        # Exponentiation by squaring algorithm
        while exponent > 0:
            if exponent % 2 == 1: # If the current bit of the exponent is 1
                result = result * base # Multiply result by the current base power
            base = base * base       # Square the base
            exponent //= 2           # Integer division by 2 (shift exponent right)

        return result

    # String representation for debugging and printing
    def __repr__(self):
        return f"M3Element(value={self.value}, system_id={id(self.system)})" # Added system_id for clarity

    # User-friendly text representation
    def text(self):
        return str(self.value)

# --- Example Usage ---

# 1. Define the specific M3System with its parameters and modulus
#    These are the fixed parameters for your algebraic structure
my_system = M3System(A=1, B=1, C=1, D=1, E=1, modulus=17)
print(f"Defined System: {my_system}\n")

# 2. Create M3Element instances within this system
a = M3Element([1, 2, 3], my_system)
e = M3Element([0, 0, 0], my_system) # The neutral element in this system

print(f"Base vector 'a' = {a.text()}")
print(f"Neutral element 'e' = {e.text()}")

# Test the neutral element property: a * e = a and e * a = a
print(f"a * e = {(a * e).text()}") # Should be [1, 2, 3]
print(f"e * a = {(e * a).text()}\n") # Should be [1, 2, 3]

# Test powers a^n
print(f"a^2 (a * a) = {(a * a).text()}")
print(f"a^2 (a**2)  = {(a**2).text()}")
print(f"a^3 (a**3)  = {(a**3).text()}")
print(f"a^4 (a**4)  = {(a**4).text()}\n")

# Test power associativity: (a*a)*(a*a) == (a*a*a)*a == a^4
print(f"Test Power Associativity:")
print(f"((a*a)*(a*a)) = {((a*a)*(a*a)).text()}") # Should be a^4
print(f"((a*a*a)*a) = {((a*a*a)*a).text()}")     # Should be a^4
print(f"a^4 from __pow__ = {(a**4).text()}\n")

# Test internal commutativity: a^m * a^n = a^n * a^m = a^(m+n)
m_exp = 2
n_exp = 3
print(f"Test Internal Commutativity (m={m_exp}, n={n_exp}):")
print(f"a^m * a^n = {(a**m_exp * a**n_exp).text()}") # a^2 * a^3
print(f"a^n * a^m = {(a**n_exp * a**m_exp).text()}") # a^3 * a^2
print(f"a^(m+n)   = {(a**(m_exp + n_exp)).text()}\n") # a^5

# Example of a Diffie-Hellman-like key exchange
print("--- Diffie-Hellman-like Key Exchange Example ---")
# Publicly agreed base vector (from the system)
public_a = M3Element([5, 8, 12], my_system) 
print(f"Public base 'a': {public_a.text()}")

# Alice's secret exponent
alice_secret_m = 7
# Alice computes and sends A = a^m
A_alice = public_a ** alice_secret_m
print(f"Alice's public A: {A_alice.text()}")

# Bob's secret exponent
bob_secret_n = 11
# Bob computes and sends B = a^n
B_bob = public_a ** bob_secret_n
print(f"Bob's public B: {B_bob.text()}")

# Alice computes shared key K_alice = B^m
K_alice = B_bob ** alice_secret_m
print(f"Alice's computed key (B^m): {K_alice.text()}")

# Bob computes shared key K_bob = A^n
K_bob = A_alice ** bob_secret_n
print(f"Bob's computed key (A^n): {K_bob.text()}")

# Verify if keys match (they should, due to power associativity and internal commutativity)
print(f"Keys match: {K_alice.value == K_bob.value}")
print(f"Expected key (a^(m*n)): {(public_a ** (alice_secret_m * bob_secret_n)).text()}")

# Test with different parameters (create a new system instance)
print("\n--- Testing a different system ---")
another_system = M3System(A=2, B=3, C=4, D=5, E=6, modulus=19)
print(f"Defined System: {another_system}")
b = M3Element([1, 2, 3], another_system)
print(f"b = {b.text()}")
print(f"b^2 = {(b**2).text()}")

Defined System: M3System(A=1, B=1, C=1, D=1, E=1, modulus=17)

Base vector 'a' = [1, 2, 3]
Neutral element 'e' = [0, 0, 0]
a * e = [1, 2, 3]
e * a = [1, 2, 3]

a^2 (a * a) = [5, 1, 10]
a^2 (a**2)  = [5, 1, 10]
a^3 (a**3)  = [12, 2, 3]
a^4 (a**4)  = [10, 6, 9]

Test Power Associativity:
((a*a)*(a*a)) = [10, 6, 9]
((a*a*a)*a) = [10, 6, 9]
a^4 from __pow__ = [10, 6, 9]

Test Internal Commutativity (m=2, n=3):
a^m * a^n = [10, 13, 11]
a^n * a^m = [10, 13, 11]
a^(m+n)   = [10, 13, 11]

--- Diffie-Hellman-like Key Exchange Example ---
Public base 'a': [5, 8, 12]
Alice's public A: [8, 7, 2]
Bob's public B: [8, 6, 9]
Alice's computed key (B^m): [2, 7, 2]
Bob's computed key (A^n): [2, 7, 2]
Keys match: True
Expected key (a^(m*n)): [2, 7, 2]

--- Testing a different system ---
Defined System: M3System(A=2, B=3, C=4, D=5, E=6, modulus=19)
b = [1, 2, 3]
b^2 = [5, 7, 1]
